#Différentes approches de sélection des fonctionnalités

##Méthodes de sélection de fonctionnalités non supervisées

Le transformateur `sklearn.feature_selection.VarianceThreshold` supprimera par défaut toutes les fonctionnalités à variance nulle. On peut également passer un seuil en argument pour lui faire supprimer les entités dont la variance est inférieure au seuil.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.05)
X_selection = sel.fit_transform(X)

Afin de supprimer les colonnes avec des valeurs manquantes, la méthode `.dropna(axis=1)` de pandas peut être utilisée sur le bloc de données.

In [ ]:
X_selection = X.dropna(axis=1)

Pour supprimer les entités à forte multicolinéarité, nous devons d’abord la mesurer. Une mesure de multicolinéarité populaire est le Variance Inflation Factor ou VIF. Il est implémenté dans le package statsmodels.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_scores = [variance_inflation_factor(X.values, feature)for feature in range(len(X.columns))]

Par convention, les colonnes avec un VIF supérieur à 10 sont considérées comme souffrant de multicolinéarité, mais un autre seuil peut être choisi s'il paraît plus raisonnable.

##Méthodes de sélection des fonctionnalités du wrapper(supervivisé):

Chaque nouveau sous-ensemble est utilisé pour entraîner un modèle dont les performances sont ensuite évaluées sur un ensemble d'exclusion. Le sous-ensemble de fonctionnalités qui donne les meilleures performances du modèle est sélectionné.

Mais en même temps, elle présente une limite. Les méthodes wrapper sont susceptibles de s'adapter au type de modèle, et les sous-ensembles de fonctionnalités qu'elles produisent pourraient ne pas se généraliser si l'on souhaite les essayer avec un modèle différent.

Un autre inconvénient important des méthodes wrapper réside dans leurs besoins de calcul importants. Ils nécessitent la formation d’un grand nombre de modèles, ce qui peut nécessiter du temps et de la puissance de calcul.

*Les méthodes d'emballage populaires incluent :

Sélection descendante , dans laquelle nous commençons par un modèle complet comprenant toutes les fonctionnalités disponibles. Dans les itérations suivantes, nous supprimons une fonctionnalité à la fois, toujours celle qui génère le gain le plus important dans une métrique de performances de modèle, jusqu'à ce que nous atteignions le nombre souhaité de fonctionnalités.
La sélection directe , qui fonctionne dans le sens inverse : on part d'un modèle nul avec zéro fonctionnalité et on les ajoute goulûment une à la fois pour maximiser les performances du modèle.

Recursive Feature Elimination , ou RFE, dont l'esprit est similaire à la sélection arrière. Il commence également par un modèle complet et élimine de manière itérative les fonctionnalités une par une. La différence réside dans la manière dont les fonctionnalités à supprimer sont choisies. Au lieu de s'appuyer sur une mesure de performance du modèle issue d'un ensemble d'éléments retenus, RFE prend sa décision en fonction de l'importance des fonctionnalités extraites du modèle. Il peut s'agir de la pondération des caractéristiques dans les modèles linéaires, de la diminution des impuretés dans les modèles arborescents ou de l'importance de la permutation (qui s'applique à tout type de modèle).*

Méthodes d'emballage en pratique

La sélection de fonctionnalités en arrière et en avant peut être implémentée avec le transformateur SequentialFeatureSelector. Par exemple, afin d'utiliser le classificateur k-Nearest-Neighbour comme modèle de notation dans la sélection directe, nous pourrions utiliser l'extrait de code suivant :

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=3, direction=”forward”)
sfs.fit(X, y)
X_selection = sfs.transform(X)

L'élimination récursive des fonctionnalités est implémentée de manière très similaire. Voici un extrait implémentant RFE basé sur l'importance des fonctionnalités d'un classificateur de vecteurs de support.

In [ ]:
from sklearn.feature_selection import RFE

svc = SVC(kernel="linear")
rfe = RFE(svc, n_features_to_select=3)
rfe.fit(X, y)
X_selection = rfe.transform(X)

#Filtrer les méthodes de sélection des fonctionnalités(supervisés)

Un autre membre de la famille supervisée est celui des méthodes de filtrage. Ils peuvent être considérés comme une alternative plus simple et plus rapide aux wrappers. Afin d'évaluer l'utilité de chaque fonctionnalité, ils analysent simplement sa relation statistique avec la cible du modèle, en utilisant des mesures telles que la corrélation ou l'information mutuelle comme indicateur de performance du modèle.

Non seulement les méthodes de filtrage sont plus rapides que les wrappers, mais elles sont également plus générales car elles sont indépendantes du modèle ; ils ne seront pas suradaptés à un algorithme particulier. Ils sont également assez faciles à interpréter : une caractéristique est ignorée si elle n’a aucune relation statistique avec la cible.

D’un autre côté, les méthodes de filtrage présentent cependant un inconvénient majeur. Ils examinent chaque fonctionnalité isolément, évaluant sa relation avec la cible. Cela les rend enclins à ignorer les fonctionnalités utiles qui sont en elles-mêmes de faibles prédicteurs de la cible, mais qui ajoutent beaucoup de valeur au modèle lorsqu'elles sont combinées avec d'autres fonctionnalités.

###Méthodes de filtrage en pratique


Voyons maintenant l'implémentation de différentes méthodes de filtrage. Ceux-ci auront besoin de plus de code de colle à implémenter. Tout d’abord, nous devons calculer la mesure de corrélation souhaitée entre chaque caractéristique et la cible. Ensuite, nous trierions toutes les fonctionnalités en fonction des résultats et conserverions le nombre souhaité (top-K ou top-30%) de celles ayant la corrélation la plus forte. Heureusement, scikit-learn fournit quelques utilitaires pour vous aider dans cette entreprise.

Pour conserver les 2 principales fonctionnalités présentant la corrélation de Pearson la plus forte avec la cible, nous pouvons exécuter :

In [ ]:
from sklearn.feature_selection import r_regression, SelectKBest

X_selection = SelectKBest(r_regression, k=2).fit_transform(X, y)

De même, pour conserver les 30 % de fonctionnalités les plus performantes, nous exécuterions :

In [ ]:
	from sklearn.feature_selection import r_regression, SelectPercentile

	X_selection = SelectPercentile(r_regression, percentile=30).fit_transform(X, y)

Les méthodes `SelectKBest` et `SelectPercentile` fonctionneront également avec des mesures de corrélation personnalisées ou non scikit-learn, à condition qu'elles renvoient un vecteur de longueur égale au nombre de fonctionnalités, avec un nombre pour chaque fonctionnalité indiquant la force de son association avec la cible. Voyons maintenant comment calculer toutes les différentes mesures de corrélation (nous discuterons de ce qu'elles signifient et quand choisir laquelle plus tard).

Spearman's Rho, Kendall Tau et la corrélation point-bisériale sont tous disponibles dans le package scipy. Voici comment obtenir leurs valeurs pour chaque fonctionnalité dans X.

In [ ]:
from scipy import stats

rho_corr = [stats.spearmanr(X[:, f], y).correlation for f in range(X.shape[1])]

tau_corr = [stats.kendalltau(X[:, f], y).correlation for f in range(X.shape[1])]

pbs_corr = [stats.pointbiserialr(X[:, f], y).correlation for f in range(X.shape[1])]

Le chi carré, les informations mutuelles et le score ANOVA F sont tous dans scikit-learn. Notez que les informations mutuelles ont une implémentation distincte, selon que la cible est nominale ou non.

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif

chi2_corr = chi2(X, y)[0]
f_corr = f_classif(X, y)[0]
mi_reg_corr = mutual_info_regression(X, y)
mi_class_corr = mutual_info_classif(X, y)

Cramer's V peut être obtenu à partir d'une version récente de Scipy (1.7.0 ou supérieure).

In [ ]:
from scipy.stats.contingency import association

v_corr = [association(np.hstack([X[:, f].reshape(-1, 1), y.reshape(-1, 1)]), method="cramer") for f in range(X.shape[1])]


##Mesurer les corrélations pour différents types de données
Lorsque les deux variables que nous comparons, c'est-à-dire la caractéristique et la cible, sont toutes deux soit un intervalle, soit un rapport, nous sommes autorisés à utiliser la mesure de corrélation la plus populaire : la corrélation de Pearson, également connue sous le nom de r de Pearson .

C’est formidable, mais la corrélation de Pearson présente deux inconvénients : elle suppose que les deux variables sont normalement distribuées et elle ne mesure que la corrélation linéaire entre elles. Lorsque la corrélation est non linéaire, le r de Pearson ne la détectera pas, même si elle est très forte.

Vous avez peut-être entendu parler de l' ensemble de données Datasaurus compilé par Alberto Cairo. Il se compose de 13 paires de variables, chacune présentant la même très faible corrélation de Pearson de -0,06. Comme cela devient rapidement évident une fois que nous les avons tracés, les paires sont en fait assez fortement corrélées, bien que de manière non linéaire.

Lorsque des relations non linéaires sont attendues, une des alternatives à la corrélation de Pearson doit être prise en compte. Les deux plus populaires sont :

Corrélation des rangs de Spearman (Spearman's Rho),
La corrélation de rang de Spearman est une alternative à la corrélation de Pearson pour les variables rapport/intervalle. Comme son nom l'indique, il examine uniquement les valeurs de classement, c'est-à-dire qu'il compare les deux variables en termes de positions relatives de points de données particuliers au sein des variables. Il est capable de capturer des relations non linéaires, mais il n'y a rien de gratuit : nous perdons certaines informations en ne considérant que le classement au lieu des points de données exacts.

Corrélation des rangs de Kendall (Kendall Tau).
Une autre mesure de corrélation basée sur les classements est la corrélation des classements de Kendall. Elle est similaire dans son esprit à la corrélation de Spearman mais formulée d'une manière légèrement différente (les calculs de Kendall sont basés sur des paires de valeurs concordantes et discordantes, par opposition aux calculs de Spearman basés sur des écarts). Kendall est souvent considéré comme plus robuste face aux valeurs aberrantes des données.

Si au moins une des variables comparées est de type ordinal, la corrélation de rang de Spearman ou de Kendall est la voie à suivre. Étant donné que les données ordinales contiennent uniquement des informations sur les rangs, elles correspondent toutes deux parfaitement, tandis que la corrélation linéaire de Pearson est de peu d'utilité.

Un autre scénario est celui où les deux variables sont nominales. Dans ce cas, nous pouvons choisir parmi plusieurs mesures de corrélation différentes :

Le V de Cramer , qui capture l'association entre les deux variables en un nombre allant de zéro (pas d'association) à un (une variable complètement déterminée par l'autre).

Statistique du chi carré couramment utilisée pour tester la dépendance entre deux variables. Le manque de dépendance suggère que la fonctionnalité particulière n’est pas utile.

Information mutuelle : mesure de dépendance mutuelle entre deux variables qui cherche à quantifier la quantité d'informations que l'on peut extraire d'une variable sur l'autre.

Dans de tels cas, les deux mesures de corrélation les plus utilisées sont :

ANOVA F-score , un équivalent du chi carré pour le cas où l'une des variables est continue tandis que l'autre est nominale,

Corrélation point-bisériale, mesure de corrélation spécialement conçue pour évaluer la relation entre une variable binaire et une variable continue.

Encore une fois, il n’existe pas de solution miracle. Le score F ne capture que les relations linéaires, tandis que la corrélation point-bisérielle émet une forte hypothèse de normalité qui pourrait ne pas être vérifiée dans la pratique, ce qui compromettrait ses résultats.